In [1]:
import numpy as np
import matplotlib.pyplot as plt
import desc.io
from desc.grid import LinearGrid, ConcentricGrid
from desc.objectives import (
    ObjectiveFunction,
    FixBoundaryR,
    FixBoundaryZ,
    FixModeR,
    FixModeZ,
    FixPressure,
    FixCurrent,
    FixIota,
    FixPsi,
    AspectRatio,
    ForceBalance,
    QuasisymmetryTwoTerm,
    RotationalTransform
)
from desc.optimize import Optimizer
from desc.plotting import (
    plot_grid,
    plot_boozer_modes,
    plot_boozer_surface,
    plot_qs_error,
    plot_boundaries,
    plot_boundary,
    plot_3d
)

from desc.equilibrium import Equilibrium
from desc.geometry import FourierRZToroidalSurface
from desc.profiles import PowerSeriesProfile

Using device: CPU, with 3.48 GB available memory


In [2]:
eq_fam = desc.io.load("inputs/lpQA.h5")
eq_init = eq_fam[-1]
eq_qs = eq_init.copy()

grid_vol = LinearGrid(
    M=eq_qs.M_grid,
    N=eq_qs.N_grid,
    NFP=eq_qs.NFP,
    rho=np.arange(0.1,1.1,0.1),
    sym=eq_qs.sym
)

In [3]:
eq_qs = eq_init.copy()
optimizer = Optimizer("Turbo1")

for maxMode in range(2,3):
    maxModeInt = maxMode
    fixed_R_modes = np.vstack(([0,0,0], eq_qs.surface.R_basis.modes[np.max(np.abs(eq_qs.surface.R_basis.modes), 1) > maxMode, :]))
    fixed_Z_modes = eq_qs.surface.Z_basis.modes[np.max(np.abs(eq_qs.surface.Z_basis.modes), 1) > maxMode, :]
    fixed_R_modes_int = eq_qs.R_basis.modes[np.max(np.abs(eq_qs.R_basis.modes), 1) > maxModeInt, :]
    fixed_Z_modes_int = eq_qs.Z_basis.modes[np.max(np.abs(eq_qs.Z_basis.modes), 1) > maxModeInt, :]
    constraints = (
        FixBoundaryR(eq=eq_qs, modes=fixed_R_modes),
        FixBoundaryZ(eq=eq_qs, modes=fixed_Z_modes),
        FixModeR(eq=eq_qs, modes=fixed_R_modes_int),
        FixModeZ(eq=eq_qs, modes=fixed_Z_modes_int),
        FixPressure(eq=eq_qs),
        FixPsi(eq=eq_qs),
        FixCurrent(eq=eq_qs),
    )   
    
    objective_f = ObjectiveFunction((
        AspectRatio(eq=eq_qs, target=6, weight=10),
        ForceBalance(eq=eq_qs, weight=1),
        RotationalTransform(eq=eq_qs, target=0.4, weight=10),
    ))
    
    global_eq_qs, result = eq_qs.optimize(
        objective=objective_f,
        constraints=constraints,
        optimizer=optimizer,
        copy=True,
        verbose=2,
        maxiter=200,
        options={"max_time":60, "trust_regions":1, "box_size":1, "training_steps": 30, "batch_size":50}
    )

/home/ryc/DESC/desc/utils.py:572: UserWarning: Detected no equilibrium constraints passed, equilibrium optimization problems usually require equilibrium constraints in order to produce meaningful, physical results. Consider adding `ForceBalance` as a constraint.
  warnings.warn(msg, err)


Building objective: aspect ratio
Precomputing transforms
Timer: Precomputing transforms = 2.90 sec
Building objective: force
Precomputing transforms
Timer: Precomputing transforms = 1.80 sec
Building objective: rotational transform
Precomputing transforms
Timer: Precomputing transforms = 2.20 sec
Timer: Objective build = 8.27 sec
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fix mode R
Building objective: fix mode Z
Building objective: fixed pressure
Building objective: fixed Psi
Building objective: fixed current
Building objective: self_consistency R
Building objective: self_consistency Z
Building objective: lambda gauge
Building objective: axis R self consistency
Building objective: axis Z self consistency
Timer: Objective build = 2.33 sec
Timer: LinearConstraintProjection build = 7.50 sec
Number of parameters: 69
Number of objectives: 544
Timer: Initializing the optimization = 18.1 sec

Starting optimization
Using method: Turbo1
Using dtype = torch.float6

KeyboardInterrupt: 

In [ ]:
plot_boundary(global_eq_qs, figsize=(6,6))
plot_boozer_surface(global_eq_qs, figsize=(6,6))
plot_3d(global_eq_qs, "|B|", figsize=(6,6))